In [ ]:
#get.valueserp.com
from functions import *
#add checklist pre-registration, power, open-data, hypotheses, research question
# article_file = "data/compassion.pdf"
# csv_name = "compassion_full.csv"
# added_lit_file = "compassion_addon.csv"

###LITERATURE
# lit_df = generate_literature_csv_from_pdf(article_file, "Author", "Title", "Publication Year", output_path=csv_name)
# if added_lit_file:
#     added_lit_df = pd.read_csv(added_lit_file, keep_default_na=False, encoding="utf-8")
#     combined_df = pd.concat([added_lit_df, lit_df], ignore_index=True) #suggest to put df with more added abstracts first
#     combined_df = remove_duplicate_rows(combined_df)
#     combined_df.to_csv("compassion_full.csv", index=False)
# df = autosearch_missing_abstracts(csv_name, csv_name)
# df = manual_entry_abstracts(csv_name, csv_name)
# data_formatted = format_texts_from_csv(csv_name, "Abstract Note", "Author", "Title", "Publication Year")
# data_formatted.to_csv("trash.csv")
# vectorstore_folder_name = generate_vectorstore(data_formatted, csv_name, max_doc_size = 4000)

###CLAIMS
# claims = extract_focal_claims(article_file, max_claims=3)
# sections = analyze_paragraphs(article_file, max_paragraphs=4)

###OPPOSITION
# relevant_docs = get_relevant_docs(presented_claim, vectorstore_folder_name)
# responses, summary_response = generate_adversarial_texts(presented_claim, relevant_docs)

In [ ]:
import requests
import json

# set up the request parameters
params = {
    'api_key': 'demo',
    'q': 'pizza',
    'location': 'London,England,United Kingdom'}
api_result = requests.get('https://api.valueserp.com/search', params).json()
print(api_result["request_info"]["success"])
print(len(api_result["organic_results"]))

In [1]:
from functions import review #"../../data/sci_rep_addon.pdf",
# from ai_oppose.functions import review
review("../../data/sci_rep_submission.pdf", "../../data/test.pdf", manual_abstracts=False, openai_model="gpt-4", max_secondary=20)


---extracting references from ../../data/sci_rep_submission.pdf

TIP: The arguments upper (~763), lower (~70), and left (~10) can trim away headers, footers, or line numbers from pdf. 
Play with trim_pdf('my_file.pdf', upper, lower, left, verbose) to test it out.
  --> 45 refs extracted

---searching for additional papers based on Abstract

serping with query : paper "critiques on gpt-3's semantic space personality prediction"
True
urls: ['https://www.nature.com/articles/s41598-023-32248-6', 'https://osf.io/89hx6/download']
ptext:: Abstract
Modern large language models generate texts that are virtually indistinguishable from those written by humans and achieve near-human performance in comprehension and reasoning tests. Yet, their complexity makes it difficult to explain and predict their functioning. We examined a state-of-the-art language model (GPT-3) using lexical decision tasks widely used to study the structure of semantic memory in humans. The results of four analyses showed th

100%|██████████| 1/1 [00:00<?, ?it/s]



---finding abstracts of related papers


---extracting references from ../../data/test.pdf

TIP: The arguments upper (~763), lower (~70), and left (~10) can trim away headers, footers, or line numbers from pdf. 
Play with trim_pdf('my_file.pdf', upper, lower, left, verbose) to test it out.
  --> 3 refs extracted

---searching for additional papers based on Abstract

serping with query : paper "critiques on self-compassion and compassion correlation studies"
True
urls: ['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5770484/', 'https://www.annualreviews.org/doi/10.1146/annurev-psych-032420-031047']
ptext:: Compassion for others and self-compassion are assumed to be closely related concepts. Yet, as they have been mostly studied separately, little is known about their relationship and to what extent they differ or resemble each other with respect to their correlates. This cross-sectional study aimed to gain knowledge on their mean levels, interrelationship, and relationships to psycholog

100%|██████████| 3/3 [00:00<00:00, 185.88it/s]



---finding abstracts of related papers


---auto-web-browsing for missing data

37 abstracts missing out of 66

m. back, s. nestler: Accuracy of judging personality
scholar found paper: True
ptext:: 
here here True True
failed to get abstract from scholar-based url. trying google
serping with query :  back Accuracy of judging personality
True
url: https://psycnet.apa.org/record/2016-47361-005
validated url
https://psycnet.apa.org/record/2016-47361-005
Accuracy of judging personality
back nestler
ptext:: Abstract
Judging other people's personality is a widespread social phenomenon early on in the acquaintance process. The accuracy of these interpersonal impressions colors the way we select, shape, and maintain our social environments. In this chapter, we give an overview of the state of the art in research on the accuracy of personality judgments. First, we describe and discuss existing methodological alternatives (variable- vs. person-centered approaches, choice of accuracy criteria, 

In [ ]:
from functions import *
import os
serp_key = os.environ["SERP_API_KEY"]
query = "A. Weiss, I. Deary: The Trait Approach"
out = search_with_backoff(query, num_results=1, max_retries=2, retry_delay_base=15, serp_key = serp_key)
out

In [ ]:
import pickle
sections = analyze_paragraphs(article_file, paragraph_length=1500, max_paragraphs=15, lower = 70, upper = 763, left = 10, verbose=False)
with open('sections.pkl', 'wb') as f:
    pickle.dump(sections, f)

In [ ]:
import pickle
with open('sections.pkl', 'rb') as f:
    sections = pickle.load(f)
for i in range(len(sections)):
    if sections[i]["reviewable"]:
        relevant_docs =  get_relevant_docs(sections[i], vectorstore_folder_name)
        if len(relevant_docs) > 0:
            responses, summary = generate_adversarial_texts(sections[i]["page_content"], relevant_docs, input_type="text")
            sections[i]["opposition"] = summary
            sections[i]["responses"] = [r["response"] for r in responses]
            sections[i]["references"] = [r["metadata"] for r in responses]
        else:
            sections[i]["opposition"] = "No relevant papers found."
            sections[i]["responses"] = []
            sections[i]["references"] = []
    else:
        sections[i]["opposition"] = "AI deemed section not reviewable."
        sections[i]["responses"] = []
        sections[i]["references"] = []
with open('sections_enriched.pkl', 'wb') as f:
    pickle.dump(sections, f)

In [ ]:
import pickle
with open('sections_enriched.pkl', 'rb') as f:
    sections = pickle.load(f)
create_pdf_from_dicts(sections, "trash.pdf")
import os 
os.startfile("trash.pdf")

In [ ]:
finds = get_abstract_from_google({"query": 'paper ' + 'Limitations of subjective ratings in instructional research'}, bias = [], query_only=True)

In [ ]:
from functions import *
# https://pubmed.ncbi.nlm.nih.gov/32961916/
#https://www.science.org/doi/10.1126/sciadv.adh1850
#
url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9796653/"
a = extract_abstract(url, '', '', query_only=True)
a